initial optimization was to remove columns but I forgot to bin any of the columns. I did want to keep this in as a reminder to make sure I take the time to properly clean my data before running the test train split and running the model.

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
df = application_df.drop(["EIN", "NAME", "APPLICATION_TYPE", "SPECIAL_CONSIDERATIONS", "USE_CASE"], axis=1)
df.head(5)

,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,C1000,Association,1,0,5000,1
1,Independent,C2000,Co-operative,1,1-9999,108590,1
2,CompanySponsored,C3000,Association,1,0,5000,0
3,CompanySponsored,C2000,Trust,1,10000-24999,6692,1
4,Independent,C1000,Trust,1,100000-499999,142590,1


In [4]:
categorical_columns = ['CLASSIFICATION', 'AFFILIATION', 'STATUS',
                       'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT']
df_encoded = pd.get_dummies(df, columns=categorical_columns)
df_encoded.head()

,IS_SUCCESSFUL,CLASSIFICATION_C0,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1230,CLASSIFICATION_C1234,CLASSIFICATION_C1235,CLASSIFICATION_C1236,CLASSIFICATION_C1237,CLASSIFICATION_C1238,...,ASK_AMT_1665460552,ASK_AMT_1736232349,ASK_AMT_1893400128,ASK_AMT_2264109450,ASK_AMT_2310256039,ASK_AMT_3391919220,ASK_AMT_4653011914,ASK_AMT_5591584994,ASK_AMT_8556638692,ASK_AMT_8597806340
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from keras.src.feature_column import dense_features_v2
# Split our preprocessed data into our features and target arrays
y = df_encoded['IS_SUCCESSFUL']

X = df_encoded.drop(columns = "IS_SUCCESSFUL")


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
input_features = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1131520   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1139841 (4.35 MB)
Trainable params: 1139841 (4.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
nn.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_data=(X_test_scaled, y_test))

Epoch 1/50
1715/1715 [==============================] - 30s 17ms/step - loss: 0.6402 - accuracy: 0.6612 - val_loss: 0.6107 - val_accuracy: 0.6829
Epoch 2/50
1715/1715 [==============================] - 29s 17ms/step - loss: 0.4494 - accuracy: 0.7765 - val_loss: 0.6307 - val_accuracy: 0.6812
Epoch 3/50
1715/1715 [==============================] - 29s 17ms/step - loss: 0.4350 - accuracy: 0.7786 - val_loss: 0.6316 - val_accuracy: 0.6880
Epoch 4/50
1715/1715 [==============================] - 27s 16ms/step - loss: 0.4327 - accuracy: 0.7809 - val_loss: 0.6381 - val_accuracy: 0.6821
Epoch 5/50
1715/1715 [==============================] - 27s 16ms/step - loss: 0.4322 - accuracy: 0.7801 - val_loss: 0.6390 - val_accuracy: 0.6834
Epoch 6/50
1715/1715 [==============================] - 30s 17ms/step - loss: 0.4306 - accuracy: 0.7802 - val_loss: 0.6478 - val_accuracy: 0.6827
Epoch 7/50
1715/1715 [==============================] - 29s 17ms/step - loss: 0.4301 - accuracy: 0.7805 - val_loss: 0.6692 -

In [10]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.8492 - accuracy: 0.6730 - 819ms/epoch - 4ms/step
Loss: 0.8491599559783936, Accuracy: 0.6730320453643799
